In [3]:
from aequilibrae import Project
import geopandas as gpd
import pandas as pd
import rasterio

In [4]:
from functions.raster_to_dataframe import raster_to_df
from functions.country_main_area import get_main_area

## Model place (Country)

In [5]:
model_place = 'Andorra'

## Loading project and getting country main area to be transformed into dataframe

In [6]:
model_fldr = f'../{model_place}'
pop_file = f'../population/{model_place}/ppp_2020.tif'

In [7]:
project = Project()
project.open(model_fldr)

In [8]:
main_area = get_main_area(project)
# modeling_region = gpd.GeoDataFrame(pd.DataFrame({'ids': [0], 'geometry': [main_area]}), crs="EPSG:4326") ## Ask about it

## Processing the raster image in Python 
(slow for large datasets)

In [9]:
dataset = rasterio.open(pop_file)
df = raster_to_df(dataset, main_area)

## Comparing the total vectorized population to the World Bank source

In [15]:
wb_tot_pop_df = pd.read_csv('../population/tot_pop_all_countries_WB.csv', skiprows = 4)

In [38]:
wb_source = wb_tot_pop_df.loc[wb_tot_pop_df['Country Name'] == model_place, '2020']
table_df = pd.DataFrame({'': ['Vectorized from raster', 'From WB source'], 'Total population': [int(df.population.sum()), int(wb_source)]})
table_df = table_df.groupby('').sum()
table_df

,Total population
,
From WB source,77265
Vectorized from raster,114728


## Creating a new table inside the model for the vectorized population

In [9]:
%%time
# Creating a new table inside the model and inserting the vectorized population there
conn = project.conn
df.to_sql('raw_population', conn, if_exists='replace', index=False)
conn.execute("select AddGeometryColumn( 'raw_population', 'geometry', 4326, 'POINT', 'XY', 1);")

CPU times: user 85.8 ms, sys: 12.3 ms, total: 98.2 ms
Wall time: 164 ms


## Creating point geometries from the population latitude and longitude coordinates

In [10]:
%%time
conn.execute("UPDATE raw_population SET Geometry=MakePoint(longitude, latitude, 4326)")
conn.commit()

CPU times: user 491 ms, sys: 5.02 ms, total: 496 ms
Wall time: 531 ms


In [11]:
project.close()